In [3]:
# render ai gym environment
import gym
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [4]:
env = gym.make('MountainCarContinuous-v0')

#print(dir(env))

print('env.observation_space', env.observation_space)
#print(dir(env.observation_space))
print('env.observation_space.shape', env.observation_space.shape)
print('env.observation_space.low', env.observation_space.low)
print('env.observation_space.high', env.observation_space.high)

print('env.action_space', env.action_space)
#print(dir(env.action_space))
print('env.action_space.shape', env.action_space.shape)
print('env.action_space.low', env.action_space.low)
print('env.action_space.high', env.action_space.high)

print('env.reward_range', env.reward_range)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
env.observation_space Box(2,)
env.observation_space.shape (2,)
env.observation_space.low [-1.2  -0.07]
env.observation_space.high [0.6  0.07]
env.action_space Box(1,)
env.action_space.shape (1,)
env.action_space.low [-1.]
env.action_space.high [1.]
env.reward_range (-inf, inf)


/opt/conda/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [ ]:
DDPGConfig = {
    'net' : {
            'state_size': 2,
            'action_size': 1,
            'phi_size': 16,
            'seed': 0,
        },
    'opt': {
            'actor_lr': 1e-3,
            'critic_lr': 1e-3,        
        },
    'replay_buffer': {
        'size': int(1e5),
    },
    'env': {
        'action_low': [-1],
        'action_high' [1],
    },
}

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class ActorCriticPolicy(nn.Module):

    def __init__(self, state_size, action_size, phi_size, seed):
        super(Policy, self).__init__()
        self.seed = torch.manual_seed(seed)
        
        self.state_size = state_size
        self.action_size = action_size
        #self.action_low = np.expand_dims(action_low, 0)
        #self.action_high = np.expand_dims(action_high, 0)
        self.phi_size = phi_size
        self.hidden_size = 64
        
        self.noise_std = noise_std

        self.fc1 = nn.Linear(self.state_size, self.hidden_size, bias=True)
        self.fc2 = nn.Linear(self.hidden_size, self.phi_size, bias=True)
        
        self.fc_actor = nn.Linear(self.phi_size, self.action_size, bias=True)
        self.fc_critic = nn.Linear(self.phi_size+self.action_size, 1, bias=True)
        
    def feature(self, state):
        hidden = F.relu(self.fc1(state))
        phi = F.relu(self.fc2(hidden))
        return phi
    
    def actor(self, phi):
        action = F.tanh(self.fc_actor(phi))
        return action
    
    def critic(self, phi, a):
        Q = self.fc_critic(torch.cat([phi,a], dim=1))
        return Q
    
    def forward(self, state):
        phi = self.feature(state)
        action = self.actor(phi)
        return action 
#, action_low, action_high
    
class DDPGAgent():
    
    def __init__(self, cfg):
        self.cfg = cfg
        
        self.seed = np.random.seed(cfg['net']['seed'])
        
        self.qnetwork_local = ActorCriticPolicy(cfg['net']).to(device)
        self.qnetwork_target = ActorCriticPolicy(cfg['net']).to(device)
        self.actor_optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=cfg['opt']['actor_lr'])
        self.critic_optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=cfg['opt']['actor_lr'])

        # Replay memory
        self.memory = ReplayBuffer(cfg['replay_buffer']['size'])
        # Initialize time step (for updating every UPDATE_EVERY steps)
        self.t_step = 0
        
    def act(self, state, std=0.):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        self.qnetwork_local.eval()
        with torch.no_grad():
            actions_local = self.qnetwork_local(state)
        self.qnetwork_local.train()
        
        actions_local = actions_local.cpu().data.numpy()
        noise = np.random.randn(*actions_local.shape) * std
        actions_local += noise
        actions_local = np.clip(actions_local, cfg['env']['action_low'], cfg['env']['action_high'])

        return actions_local
        
    '''
    def step(self, state, action, reward, next_state, done):        
        idx = self.memory._next_idx
        
        # Add sample buffer to replay memory
        self.memory.add(state, action, reward, next_state, done)
        states = np.expand_dims(state, axis=0)
        actions = np.expand_dims(np.expand_dims(action, axis=0), axis=-1)
        rewards = np.expand_dims(reward, axis=0)        
        next_states = np.expand_dims(next_state, axis=0)        
        dones = np.expand_dims(done, axis=0)
        
        delta = self.compute_delta(states, actions, rewards, next_states, dones, GAMMA)
        priorities = np.abs(np.squeeze(delta.cpu().detach().numpy(), axis=0)) + PRIORITIZED_REPLAY_EPS
        self.memory.update_priorities([idx], priorities)
        
        # Learn every UPDATE_EVERY time steps.
        self.t_step = (self.t_step + 1) % UPDATE_EVERY
        if self.t_step == 0:
            
          
            # If enough samples are available in memory, get random subset and learn
            if len(self.memory) > BATCH_SIZE:
                experiences = self.memory.sample(BATCH_SIZE, self.beta(done))
                self.memory.alpha = self.alpha(done)
                self.learn(experiences, GAMMA)
                               
                

    
    def compute_delta(self, states, actions, rewards, next_states, dones, gamma):
        states = torch.from_numpy(states).float().to(device)
        actions = torch.from_numpy(actions).long().to(device)
        rewards = torch.from_numpy(rewards).float().to(device)
        next_states = torch.from_numpy(next_states).float().to(device)
        dones = torch.from_numpy(dones.astype(np.uint8)).float().to(device)
        
        # Get max predicted Q values (for next states) from target model
        # Double DQN
        actions_next_state = self.qnetwork_local(states).detach().argmax(1).unsqueeze(1)
        Q_targets_next = self.qnetwork_target(next_states).detach().gather(1, actions_next_state)
        # DQN
        # Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
        # Compute Q targets for current states
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))

        # Get expected Q values from local model
        Q_expected = self.qnetwork_local(states).gather(1, actions)
        
        # Compute loss
        delta = Q_targets - Q_expected
        
        return delta

    def learn(self, experiences, gamma):
        states, actions, rewards, next_states, dones, weights, indices = experiences
        
        states = torch.from_numpy(states).float().to(device)
        actions = torch.from_numpy(np.expand_dims(actions, axis=-1)).long().to(device)
        rewards = torch.from_numpy(np.expand_dims(rewards, axis=-1)).float().to(device)
        next_states = torch.from_numpy(next_states).float().to(device)
        dones = torch.from_numpy(np.expand_dims(dones.astype(np.uint8), axis=-1)).float().to(device)        
        weights = torch.from_numpy(weights).float().to(device)

        # Get max predicted Q values (for next states) from target model
        # Double DQN
        actions_next_state = self.qnetwork_local(states).detach().argmax(1).unsqueeze(1)
        Q_targets_next = self.qnetwork_target(next_states).detach().gather(1, actions_next_state)
        # DQN
        # Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
        # Compute Q targets for current states
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))

        # Get expected Q values from local model
        Q_expected = self.qnetwork_local(states).gather(1, actions)
        
        # Compute loss
        delta = Q_targets - Q_expected
        squared_difference = delta ** 2
        weighted_squared_difference = weights * squared_difference
        loss = weighted_squared_difference.mean()
        # Minimize the loss
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # ------------------- update target network ------------------- #
        self.soft_update(self.qnetwork_local, self.qnetwork_target, TAU) 

        # ------------------- update priorities ------------------- #        
        new_priorities = np.abs(np.squeeze(delta.cpu().detach().numpy())) + PRIORITIZED_REPLAY_EPS
        self.memory.update_priorities(indices, new_priorities)
        
    def soft_update(self, local_model, target_model, tau):
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)
    '''
            
def ddpg(agent, n_episodes=2000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start                    # initialize epsilon
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name] # reset the environment
        state = env_info.vector_observations[0]            # get the current state
        score = 0
        for t in range(max_t):
            action = agent.act(state, eps)                 # select an action
            env_info = env.step(action)[brain_name]        # send the action to the environment
            next_state = env_info.vector_observations[0]   # get the next state
            reward = env_info.rewards[0]                   # get the reward
            done = env_info.local_done[0]                  # see if episode has finished       
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break 
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>=13.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
            break
    return scores

          


'    \n    def _action(self, state):\n        phi = self._phi(state)\n        \n        _actor_mean = F.sigmoid(self.actor(phi)) * (self.action_high - self.action_low) + self.action_low\n        _actor_noise = torch.randn(_actor_mean.size()) * self.noise_std\n        _actor_out = torch.clamp(_actor_mean + _actor_noise, self.action_low, self.action_high)\n        \n        return _actor_out, _actor_mean\n    \n    def _critic(self, state, action):\n        _critic_out = self.critic(torch.cat([phi, action], dim=1))\n        \n        return _critic_out\n       \n    def forward(self, state, action):\n    \n        hidden = F.relu(self.fc1(state))\n        phi = F.relu(self.fc2(hidden))\n        \n        _actor_mean = F.sigmoid(self.actor(phi)) * (self.action_high - self.action_low) + self.action_low\n        _actor_noise = torch.randn(_actor_mean.size()) * self.noise_std\n        _actor_out = torch.clamp(_actor_mean + _actor_noise, self.action_low, self.action_high)\n        \n        _

In [ ]:
import time
start = time.time()
scores = ddpg(n_episodes=2000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995)
end = time.time()
print('time = {:.2f} s'.format(end-start))  

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

def dqn(n_episodes=2000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start                    # initialize epsilon
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name] # reset the environment
        state = env_info.vector_observations[0]            # get the current state
        score = 0
        for t in range(max_t):
            action = agent.act(state, eps)                 # select an action
            env_info = env.step(action)[brain_name]        # send the action to the environment
            next_state = env_info.vector_observations[0]   # get the next state
            reward = env_info.rewards[0]                   # get the reward
            done = env_info.local_done[0]                  # see if episode has finished       
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break 
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>=13.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
            break
    return scores

import time
start = time.time()
scores = dqn(n_episodes=2000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995)
end = time.time()
print('time = {:.2f} s'.format(end-start))

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')

# Close the environment
env.close()

In [2]:
# training loop max iterations
episode = 500

# widget bar to display progress
!pip install progressbar
import progressbar as pb
widget = ['training loop: ', pb.Percentage(), ' ', 
          pb.Bar(), ' ', pb.ETA() ]
timer = pb.ProgressBar(widgets=widget, maxval=episode).start()

# keep track of progress
mean_rewards = []

for e in range(episode):

    # collect trajectories
    old_probs, states, actions, rewards = \
        pong_utils.collect_trajectories(envs, policy, tmax=tmax)
        
    total_rewards = np.sum(rewards, axis=0)


    # gradient ascent step
    for _ in range(SGD_epoch):
        
        # uncomment to utilize your own clipped function!
        L = -clipped_surrogate(policy, old_probs, states, actions, rewards, epsilon=epsilon, beta=beta)

        # L = -pong_utils.clipped_surrogate(policy, old_probs, states, actions, rewards,
        #                                    epsilon=epsilon, beta=beta)
        optimizer.zero_grad()
        L.backward()
        optimizer.step()
        del L
    
    # the clipping parameter reduces as time goes on
    epsilon*=.999
    
    # the regulation term also reduces
    # this reduces exploration in later runs
    beta*=.995
    
    # get the average reward of the parallel environments
    mean_rewards.append(np.mean(total_rewards))
    
    # display some progress every 20 iterations
    if (e+1)%20 ==0 :
        print("Episode: {0:d}, score: {1:f}".format(e+1,np.mean(total_rewards)))
        print(total_rewards)
        
    # update progress widget bar
    timer.update(e+1)
    
timer.finish()


  Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProxyError('Cannot connect to proxy.', NewConnectionError('<pip._vendor.urllib3.connection.VerifiedHTTPSConnection object at 0x7f56f2248550>: Failed to establish a new connection: [Errno 111] Connection refused',))': /simple/progressbar/
  Retrying (Retry(total=3, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProxyError('Cannot connect to proxy.', NewConnectionError('<pip._vendor.urllib3.connection.VerifiedHTTPSConnection object at 0x7f56f33a3668>: Failed to establish a new connection: [Errno 111] Connection refused',))': /simple/progressbar/
  Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProxyError('Cannot connect to proxy.', NewConnectionError('<pip._vendor.urllib3.connection.VerifiedHTTPSConnection object at 0x7f56f33a3128>: Failed to establish a new connection: [Errno 111] Conne

ModuleNotFoundError: No module named 'progressbar'

In [ ]:
from parallelEnv import parallelEnv
import numpy as np
# keep track of how long training takes
# WARNING: running through all 800 episodes will take 30-45 minutes

# training loop max iterations
episode = 500

# widget bar to display progress
!pip install progressbar
import progressbar as pb
widget = ['training loop: ', pb.Percentage(), ' ', 
          pb.Bar(), ' ', pb.ETA() ]
timer = pb.ProgressBar(widgets=widget, maxval=episode).start()


envs = parallelEnv('PongDeterministic-v4', n=8, seed=1234)

discount_rate = .99
epsilon = 0.1
beta = .01
tmax = 320
SGD_epoch = 4

# keep track of progress
mean_rewards = []

for e in range(episode):

    # collect trajectories
    old_probs, states, actions, rewards = \
        pong_utils.collect_trajectories(envs, policy, tmax=tmax)
        
    total_rewards = np.sum(rewards, axis=0)


    # gradient ascent step
    for _ in range(SGD_epoch):
        
        # uncomment to utilize your own clipped function!
        L = -clipped_surrogate(policy, old_probs, states, actions, rewards, epsilon=epsilon, beta=beta)

        # L = -pong_utils.clipped_surrogate(policy, old_probs, states, actions, rewards,
        #                                    epsilon=epsilon, beta=beta)
        optimizer.zero_grad()
        L.backward()
        optimizer.step()
        del L
    
    # the clipping parameter reduces as time goes on
    epsilon*=.999
    
    # the regulation term also reduces
    # this reduces exploration in later runs
    beta*=.995
    
    # get the average reward of the parallel environments
    mean_rewards.append(np.mean(total_rewards))
    
    # display some progress every 20 iterations
    if (e+1)%20 ==0 :
        print("Episode: {0:d}, score: {1:f}".format(e+1,np.mean(total_rewards)))
        print(total_rewards)
        
    # update progress widget bar
    timer.update(e+1)
    
timer.finish()

In [ ]:
env.close()

In [ ]:
            
'''    
    def _action(self, state):
        phi = self._phi(state)
        
        _actor_mean = F.sigmoid(self.actor(phi)) * (self.action_high - self.action_low) + self.action_low
        _actor_noise = torch.randn(_actor_mean.size()) * self.noise_std
        _actor_out = torch.clamp(_actor_mean + _actor_noise, self.action_low, self.action_high)
        
        return _actor_out, _actor_mean
    
    def _critic(self, state, action):
        _critic_out = self.critic(torch.cat([phi, action], dim=1))
        
        return _critic_out
       
    def forward(self, state, action):
    
        hidden = F.relu(self.fc1(state))
        phi = F.relu(self.fc2(hidden))
        
        _actor_mean = F.sigmoid(self.actor(phi)) * (self.action_high - self.action_low) + self.action_low
        _actor_noise = torch.randn(_actor_mean.size()) * self.noise_std
        _actor_out = torch.clamp(_actor_mean + _actor_noise, self.action_low, self.action_high)
        
        _critic_out = self.critic(phi)        

        return x
    

   '''